In [14]:
import pyodbc
from meyerDB import cable_connection
import PyQt5
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
from IPython.display import HTML, display, clear_output

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeavePOut
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_absolute_error as mae
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
import sklearn.preprocessing as pp
from scipy.optimize import curve_fit
 

# init db connection
conn = pyodbc.connect(cable_connection)
cursor = conn.cursor()
print('Database connection ok')

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            try:
                value = str(round(100*field, 1)).replace('.', ',') + '%'
            except:
                value = field
            html += "<td><h4>%s</h4><td>"%(value)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

def logifunc(x,x0,k,l, A):
    #l = 2300
    return l / (1 + A*np.exp(-k*(x - x0)))

Database connection ok


In [20]:
# Design progress
# Predictors:
# Time series
# Algorithms:
# S-curve fitting


plt.rcParams['font.size'] = 14
plt.rcParams['font.weight'] = 'bold'

# Get the data
cursor.execute("SELECT DISTINCT project_id FROM routed")
data = np.array(cursor.fetchall()).astype('int32')
ships = data[:, 0]


# init lpo split
p = 2 #ships
lpo = LeavePOut(p)


wk = 'pw'
cursor.execute("UPDATE progress SET cables=0")
cursor.execute(
    "UPDATE progress SET progress.cables=t1.cables FROM"
    " (SELECT project_id, {}, sum(amount) as cables FROM ship_readiness GROUP BY project_id, {}) t1"
    " WHERE t1.project_id=progress.project_id AND t1.{}=progress.wk".format(wk, wk, wk)
)
cursor.commit()

ship_data = {}
for ship in ships:
    cursor.execute("SELECT wk, cables FROM progress WHERE project_id={} and cables > 100 ORDER BY wk".format(ship))
    data = cursor.fetchall()
    ship_data[ship] = np.array(data)

for ship in ships:
    x = ship_data[ship][:, 0]
    y = ship_data[ship][:, 1].cumsum()
    goal = y[-1]
    for wk in np.arange(-100, 1):
        try:
            cut = np.argwhere(x <= wk)[-1][0] + 1
        except IndexError:
            continue
        xx = x[:cut]
        yy = y[:cut]
        if (goal - yy[-1]) < 0.01*goal: continue
        plt.plot([-100, 0], [goal, goal])
        try:
            params, _ = curve_fit(logifunc, xx, yy, bounds=([-100, 1e-3, goal-1, 1e-4], [-10, 2, goal+1, 100]))
        except ValueError:
            continue

        clear_output(wait=True)
        plt.plot(x, y, 'kx')
        plt.plot(xx, yy, 'rx')
        plt.plot(x[cut:], logifunc(x[cut:], *params))
        plt.ylim([0, 3e6])
        plt.pause(0.1)
        plt.cla()
        



C:\Users\sainmi53\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
